In [1]:
# Libraries
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import warnings

from sklearn.metrics         import mean_squared_error
from sklearn.ensemble        import RandomForestRegressor
from sklearn.ensemble        import AdaBoostRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression 


warnings.filterwarnings('ignore')

In [2]:
# local file paths

dir_name = 'selezione'
region_names = np.array(['A', 'B', 'C'])

fp_Xtrain = []
fp_Xval   = []
fp_Xtest  = []
fp_ytrain = []
fp_yval   = []
fp_ytest  = []

for i in range(3):
    fp_Xtrain.append(dir_name + f'/X_train{region_names[i]}.csv')
    fp_Xval  .append(dir_name + f'/X_val{  region_names[i]}.csv')
    fp_Xtest .append(dir_name + f'/X_test{ region_names[i]}.csv')
    fp_ytrain.append(dir_name + f'/y_train{region_names[i]}.csv')
    fp_yval  .append(dir_name + f'/y_val{  region_names[i]}.csv')
    fp_ytest .append(dir_name + f'/y_test{ region_names[i]}.csv')

In [3]:
# Lettura dei dati

X_train = []
X_val   = []
X_test  = []
y_train = []
y_val   = []
y_test  = []

for i in range(3):
    X_train.append(pd.read_csv(fp_Xtrain[i], low_memory=False))
    X_val  .append(pd.read_csv(fp_Xval  [i], low_memory=False))        
    X_test .append(pd.read_csv(fp_Xtest [i], low_memory=False))
    y_train.append(pd.read_csv(fp_ytrain[i], low_memory=False))
    y_val  .append(pd.read_csv(fp_yval  [i], low_memory=False))        
    y_test .append(pd.read_csv(fp_ytest [i], low_memory=False))
    
X_train = np.array(X_train, dtype=object)
X_val   = np.array(X_val,   dtype=object)
X_test  = np.array(X_test,  dtype=object)
y_train = np.array(y_train, dtype=object)
y_val   = np.array(y_val,   dtype=object)
y_test  = np.array(y_test,  dtype=object)

In [4]:
def dimensionality(y=False):
    for i in range(3):
        print(f'X_train{region_names[i]}: {X_train[i].shape}')
        print(f'X_val{region_names[i]}:   {X_val  [i].shape}')
        print(f'X_test{region_names[i]}:  {X_test [i].shape}')
        if y:
            print(f'y_train{region_names[i]}: {y_train[i].shape}')
            print(f'y_val{region_names[i]}:   {y_val  [i].shape}')
            print(f'y_test{region_names[i]}:  {y_test [i].shape}')
            print()

In [5]:
dimensionality(y=True)

X_trainA: (26819, 41)
X_valA:   (9006, 41)
X_testA:  (9085, 41)
y_trainA: (26819, 1)
y_valA:   (9006, 1)
y_testA:  (9085, 1)

X_trainB: (8119, 34)
X_valB:   (2658, 34)
X_testB:  (2606, 34)
y_trainB: (8119, 1)
y_valB:   (2658, 1)
y_testB:  (2606, 1)

X_trainC: (64771, 48)
X_valC:   (21908, 48)
X_testC:  (21876, 48)
y_trainC: (64771, 1)
y_valC:   (21908, 1)
y_testC:  (21876, 1)



In [6]:
dt_model = np.array([
    DecisionTreeRegressor(max_leaf_nodes = 2),
    DecisionTreeRegressor(max_leaf_nodes = 2),
    DecisionTreeRegressor(max_leaf_nodes = 2)
])

In [7]:
boost_model = np.array([
    AdaBoostRegressor(dt_model[0], n_estimators = 45),
    AdaBoostRegressor(dt_model[1], n_estimators = 54),
    AdaBoostRegressor(dt_model[1], n_estimators = 37)
])

In [8]:
rf_model = np.array([
    RandomForestRegressor(n_estimators = 45, n_jobs = -1),
    RandomForestRegressor(n_estimators = 38, n_jobs = -1),
    RandomForestRegressor(n_estimators = 51, n_jobs = -1)
])

In [9]:
models = np.array([
    dt_model,
    boost_model,
    rf_model
])

In [29]:
def gen_Y(X, y, models, col_names, index):
    Y = pd.DataFrame()
    for i in range(len(col_names)):
        models[i][index].fit(X[index], y[index])
        Y[col_names[i]] = models[i][index].predict(X[index])
    return Y

In [104]:
def gen_arrY(X, y, models, col_names):
    Y = []
    for i in range(len(X)):
        Y.append(gen_Y(X, y, models, col_names, i))
    Y = np.array(Y, dtype=object)
    return Y

In [105]:
col_names = ['DecisionTree', 'Boosting', 'RandomForest']

In [106]:
Y_train = gen_arrY(X_train, y_train, models, col_names)
Y_test  = gen_arrY(X_test,  y_test,  models, col_names)

In [109]:
def dimensionalityY():
    for i in range(3):
        print(region_names[i])
        print(f'X_train{region_names[i]}: {X_train[i].shape}')
        print(f'Y_train{region_names[i]}: {Y_train[i].shape}')
        print(f'y_train{region_names[i]}: {y_train[i].shape}')
        print()
        print(f'X_test{region_names[i]}: {X_test[i].shape}')
        print(f'Y_test{region_names[i]}: {Y_test[i].shape}')
        print(f'y_test{region_names[i]}: {y_test[i].shape}')
        print()

In [110]:
dimensionalityY()

A
X_trainA: (26819, 41)
Y_trainA: (26819, 3)
y_trainA: (26819, 1)

X_testA: (9085, 41)
Y_testA: (9085, 3)
y_testA: (9085, 1)

B
X_trainB: (8119, 34)
Y_trainB: (8119, 3)
y_trainB: (8119, 1)

X_testB: (2606, 34)
Y_testB: (2606, 3)
y_testB: (2606, 1)

C
X_trainC: (64771, 48)
Y_trainC: (64771, 3)
y_trainC: (64771, 1)

X_testC: (21876, 48)
Y_testC: (21876, 3)
y_testC: (21876, 1)



In [154]:
def linear_regression_preds(Y_train, y_train, Y_test):
    preds = []
    for Y_trn, y_trn, Y_tst in zip(Y_train, y_train, Y_test):
        preds.append(
            LinearRegression(fit_intercept=True).fit(Y_trn, y_trn).predict(Y_tst)
        )
    preds = np.array(preds, dtype = object)
    return preds

In [155]:
y_preds = linear_regression_preds(Y_train, y_train, Y_test)

In [160]:
for i in range(3):
    print(region_names[i])
    print(y_preds[i].shape)
    print(y_test[i].shape)
    print()

A
(9085, 1)
(9085, 1)

B
(2606, 1)
(2606, 1)

C
(21876, 1)
(21876, 1)



In [164]:
def get_mse(y_trues, y_preds):
    mse = []
    for y_true, y_pred in zip(y_trues, y_preds):
        mse.append(mean_squared_error(y_true, y_pred))
    mse = np.array(mse, dtype = object)
    return mse

In [166]:
mse = get_mse(y_test, y_preds)

In [167]:
mse

array([0.0031581082324556573, 0.002673794167959957, 0.0018657396656370543],
      dtype=object)

In [169]:
for i in range(3):
    print(region_names[i])
    print(mse[i])    
    print()    

A
0.0031581082324556573

B
0.002673794167959957

C
0.0018657396656370543



In [254]:
def best_prediction(X, y_trues, y_preds, index, nrow=10):
    err  = abs(y_trues[index] - y_preds[index])
    err  = err.values.ravel()
    args = err.argsort()
    for i in range(nrow):
        print(f'Row: {args[i]}, error: {err[args[i]]}')
    return X[index].iloc[args[:nrow]].transpose()

In [255]:
best_prediction(X_test, y_test, y_preds, 0, nrow=5)

Row: 1608, error: 5.44889575913865e-06
Row: 3890, error: 7.6305282379098e-06
Row: 387, error: 1.0708820749050219e-05
Row: 7390, error: 1.6395960858187064e-05
Row: 5022, error: 2.0358383663155767e-05


,1608,3890,387,7390,5022
bathroomcnt,2.000000e+00,3.000000e+00,2.500000e+00,2.500000e+00,1.000000e+00
bedroomcnt,4.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00
calculatedbathnbr,2.000000e+00,3.000000e+00,2.500000e+00,2.500000e+00,1.000000e+00
calculatedfinishedsquarefeet,2.049000e+03,2.152000e+03,1.250000e+03,1.322000e+03,9.520000e+02
finishedsquarefeet12,2.049000e+03,2.152000e+03,1.250000e+03,1.322000e+03,9.520000e+02
fireplacecnt,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
latitude,3.378071e+07,3.369591e+07,3.385836e+07,3.365390e+07,3.353412e+07
longitude,1.178372e+08,1.177454e+08,1.177284e+08,1.179470e+08,1.176669e+08
lotsizesquarefeet,1.500000e+04,7.205000e+03,7.205000e+03,7.205000e+03,3.300000e+03
rawcensustractandblock,6.059076e+07,6.059052e+07,6.059022e+07,6.059064e+07,6.059032e+07


In [256]:
best_prediction(X_test, y_test, y_preds, 1, nrow=5)

Row: 1892, error: 1.2318134620020738e-05
Row: 1238, error: 2.119314641759948e-05
Row: 694, error: 4.127586140760531e-05
Row: 1237, error: 5.0205249543973374e-05
Row: 670, error: 5.541832156980009e-05


,1892,1238,694,1237,670
bathroomcnt,2.500000e+00,2.000000e+00,2.500000e+00,1.500000e+00,2.000000e+00
bedroomcnt,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00
calculatedbathnbr,2.500000e+00,2.000000e+00,2.500000e+00,1.500000e+00,2.000000e+00
calculatedfinishedsquarefeet,1.547000e+03,1.486000e+03,1.803000e+03,1.403000e+03,1.334000e+03
finishedsquarefeet12,1.547000e+03,1.486000e+03,1.803000e+03,1.403000e+03,1.334000e+03
fireplacecnt,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
latitude,3.426536e+07,3.421878e+07,3.427142e+07,3.416463e+07,3.430374e+07
longitude,1.188989e+08,1.188449e+08,1.188933e+08,1.188464e+08,1.188452e+08
lotsizesquarefeet,7.205000e+03,1.044600e+04,5.224000e+03,2.390000e+04,5.874000e+03
rawcensustractandblock,6.111008e+07,6.111008e+07,6.111008e+07,6.111006e+07,6.111008e+07


In [257]:
best_prediction(X_test, y_test, y_preds, 2, nrow=5)

Row: 14256, error: 4.7254731469015206e-06
Row: 14079, error: 1.0073447930969426e-05
Row: 1982, error: 1.0692663301468952e-05
Row: 8856, error: 1.1020492149863004e-05
Row: 5023, error: 1.2599463038345804e-05


,14256,14079,1982,8856,5023
bathroomcnt,3.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00
bedroomcnt,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00
buildingqualitytypeid,8.000000e+00,6.000000e+00,4.000000e+00,8.000000e+00,6.000000e+00
calculatedbathnbr,3.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00
calculatedfinishedsquarefeet,1.884000e+03,1.204000e+03,1.070000e+03,1.374000e+03,1.287000e+03
finishedsquarefeet12,1.884000e+03,1.204000e+03,1.070000e+03,1.374000e+03,1.287000e+03
latitude,3.423231e+07,3.419725e+07,3.379900e+07,3.381947e+07,3.414618e+07
longitude,1.186054e+08,1.185704e+08,1.183020e+08,1.183212e+08,1.183777e+08
lotsizesquarefeet,1.231500e+04,8.086000e+03,7.205000e+03,4.050400e+04,6.500000e+03
rawcensustractandblock,6.037113e+07,6.037135e+07,6.037293e+07,6.037651e+07,6.037144e+07


In [260]:
def worst_prediction(X, y_trues, y_preds, index, nrow=10):
    err  = abs(y_trues[index] - y_preds[index])
    err  = err.values.ravel()
    args = err.argsort()[::-1]
    for i in range(nrow):
        print(f'Row: {args[i]}, error: {err[args[i]]}')
    return X[index].iloc[args[:nrow]].transpose()

In [261]:
worst_prediction(X_test, y_test, y_preds, 0, nrow=5)

Row: 5541, error: 1.04798431783384
Row: 8089, error: 0.9993323882959884
Row: 4406, error: 0.8565590125797439
Row: 2918, error: 0.8271780562889401
Row: 5097, error: 0.8135126414412008


,5541,8089,4406,2918,5097
bathroomcnt,2.000000e+00,1.500000e+00,2.000000e+00,1.000000e+00,1.000000e+00
bedroomcnt,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
calculatedbathnbr,2.000000e+00,1.500000e+00,2.000000e+00,1.000000e+00,1.000000e+00
calculatedfinishedsquarefeet,1.250000e+03,1.902000e+03,1.260000e+03,1.674000e+03,1.256000e+03
finishedsquarefeet12,1.524000e+03,1.902000e+03,1.260000e+03,1.524000e+03,1.524000e+03
fireplacecnt,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
latitude,3.383566e+07,3.380082e+07,3.360190e+07,3.376907e+07,3.382855e+07
longitude,1.179094e+08,1.178508e+08,1.179010e+08,1.178658e+08,1.179484e+08
lotsizesquarefeet,7.307000e+03,6.650000e+03,7.205000e+03,1.268800e+04,1.254400e+04
rawcensustractandblock,6.059087e+07,6.059076e+07,6.059063e+07,6.059076e+07,6.059087e+07


In [262]:
worst_prediction(X_test, y_test, y_preds, 1, nrow=5)

Row: 2221, error: 0.46231316012917323
Row: 1393, error: 0.4594084165262182
Row: 298, error: 0.410917146456766
Row: 1109, error: 0.3590955388180627
Row: 2001, error: 0.32434329707233567


,2221,1393,298,1109,2001
bathroomcnt,2.500000e+00,2.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00
bedroomcnt,3.000000e+00,5.000000e+00,3.000000e+00,1.000000e+00,4.000000e+00
calculatedbathnbr,2.500000e+00,2.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00
calculatedfinishedsquarefeet,1.605000e+03,1.860000e+03,1.120000e+03,6.000000e+02,3.758000e+03
finishedsquarefeet12,1.605000e+03,1.860000e+03,1.120000e+03,6.000000e+02,3.758000e+03
fireplacecnt,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
latitude,3.425348e+07,3.422859e+07,3.427652e+07,3.416070e+07,3.413656e+07
longitude,1.188026e+08,1.191919e+08,1.187467e+08,1.192255e+08,1.188856e+08
lotsizesquarefeet,7.205000e+03,6.534000e+03,7.300000e+03,2.625000e+03,7.205000e+03
rawcensustractandblock,6.111008e+07,6.111003e+07,6.111008e+07,6.111004e+07,6.111007e+07


In [263]:
worst_prediction(X_test, y_test, y_preds, 2, nrow=5)

Row: 18590, error: 0.9289787791126285
Row: 4291, error: 0.7267326722937435
Row: 6614, error: 0.5566632198702617
Row: 5840, error: 0.5299377073740877
Row: 20980, error: 0.4795486018804156


,18590,4291,6614,5840,20980
bathroomcnt,2.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00
bedroomcnt,3.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00
buildingqualitytypeid,7.000000e+00,4.000000e+00,7.000000e+00,4.000000e+00,4.000000e+00
calculatedbathnbr,2.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00
calculatedfinishedsquarefeet,1.417000e+03,1.824000e+03,6.400000e+02,1.782000e+03,3.098000e+03
finishedsquarefeet12,1.417000e+03,1.524000e+03,6.400000e+02,1.782000e+03,3.098000e+03
latitude,3.401871e+07,3.394890e+07,3.418350e+07,3.406119e+07,3.411826e+07
longitude,1.184192e+08,1.182540e+08,1.185960e+08,1.187407e+08,1.181010e+08
lotsizesquarefeet,6.344000e+03,2.491000e+03,1.345810e+05,1.012300e+04,1.046200e+04
rawcensustractandblock,6.037272e+07,6.037241e+07,6.037135e+07,6.037800e+07,6.037464e+07
